In [1]:
text = 'Toxic shock syndrome(TSS)は黄色ぶどう球菌やA群 β 溶血性連鎖球菌が産生するスーパー抗原に起因した感染性疾患である.'

In [2]:
import spacy
nlp = spacy.load('ja_ginza')

In [3]:
doc = nlp(text)

In [4]:
len(doc)

31

In [5]:
mask_ratio = 0.15
span_length = 3

In [6]:
import math

In [7]:
def get_chunk_num(doc):
    mask_token = math.ceil(len(doc)*mask_ratio)
    chunk_num = math.ceil(mask_token/span_length)
    return chunk_num

In [8]:
chunk_num = get_chunk_num(doc)

In [9]:
chunk_num

2

In [10]:
index_sequence = [i for i in range(len(doc)-span_length+1)]
for i in range(len(index_sequence)):
    index_sequence[i] = [i for i in range(i,i+span_length)] 

In [11]:
from random import randint

In [12]:
mask_index = []
while chunk_num > 0:
    masked_spans = index_sequence[randint(0,len(index_sequence))]
    mask_index.extend(masked_spans)
    index_id = index_sequence.index(masked_spans)
    del index_sequence[index_id-span_length+1:index_id+span_length]
    chunk_num = chunk_num - 1

In [13]:
mask_index.sort()

In [14]:
mask_index

[9, 10, 11, 19, 20, 21]

In [15]:
text_sequence = [i for i in range(len(doc))]
unmask_index = list(set(text_sequence) - set(mask_index))

In [16]:
unmask_index.sort()

In [17]:
unmask_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30]

In [36]:
input_text = [None] * len(doc)
for i in unmask_index:
    input_text[i] = doc[i]

In [40]:
input_text

[Toxic,
 shock,
 syndrome,
 (,
 TSS,
 ),
 は,
 黄色,
 ぶどう,
 None,
 None,
 None,
 群,
 β,
 溶血性,
 連鎖,
 球菌,
 が,
 産生,
 None,
 None,
 None,
 に,
 起因,
 し,
 た,
 感染性,
 疾患,
 で,
 ある,
 .]

In [51]:
re_input_text = []
for i in range(len(input_text)-1):
        if str(input_text[i]) == 'None' and str(input_text[i+1]) == 'None':
            pass
        else:
            re_input_text.append(input_text[i])
        i+=2               

re_input_text.append(input_text[len(input_text)-1])

In [52]:
re_input_text

[Toxic,
 shock,
 syndrome,
 (,
 TSS,
 ),
 は,
 黄色,
 ぶどう,
 None,
 群,
 β,
 溶血性,
 連鎖,
 球菌,
 が,
 産生,
 None,
 に,
 起因,
 し,
 た,
 感染性,
 疾患,
 で,
 ある,
 .]

In [54]:
sent_token_id = 0
for i in range(len(re_input_text)):
    if str(re_input_text[i]) == 'None':
        re_input_text[i] = f'<extra_id_{sent_token_id}>'
        sent_token_id += 1    

In [55]:
re_input_text

[Toxic,
 shock,
 syndrome,
 (,
 TSS,
 ),
 は,
 黄色,
 ぶどう,
 '<extra_id_0>',
 群,
 β,
 溶血性,
 連鎖,
 球菌,
 が,
 産生,
 '<extra_id_1>',
 に,
 起因,
 し,
 た,
 感染性,
 疾患,
 で,
 ある,
 .]

In [56]:
inputs = ''.join(str(e) for e in re_input_text)

In [57]:
inputs

'Toxicshocksyndrome(TSS)は黄色ぶどう<extra_id_0>群β溶血性連鎖球菌が産生<extra_id_1>に起因した感染性疾患である.'

In [58]:
target_text = [None] * len(doc)
for i in mask_index:
    target_text[i] = doc[i]

In [59]:
target_text

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 球菌,
 や,
 A,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 する,
 スーパー,
 抗原,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [60]:
re_target_text = []
for i in range(len(target_text)-1):
        if str(target_text[i]) == 'None' and str(target_text[i+1]) == 'None':
            pass
        else:
            re_target_text.append(target_text[i])
        i+=2               

re_target_text.append(target_text[len(target_text)-1])

In [61]:
re_target_text

[None, 球菌, や, A, None, する, スーパー, 抗原, None]

In [62]:
sent_token_id = 0
for i in range(len(re_target_text)):
    if str(re_target_text[i]) == 'None':
        re_target_text[i] = f'<extra_id_{sent_token_id}>'
        sent_token_id += 1 

In [63]:
re_target_text

['<extra_id_0>', 球菌, や, A, '<extra_id_1>', する, スーパー, 抗原, '<extra_id_2>']

In [64]:
targets = ''.join(str(e) for e in re_target_text)

In [65]:
targets 

'<extra_id_0>球菌やA<extra_id_1>するスーパー抗原<extra_id_2>'

In [66]:
inputs

'Toxicshocksyndrome(TSS)は黄色ぶどう<extra_id_0>群β溶血性連鎖球菌が産生<extra_id_1>に起因した感染性疾患である.'